### Ensemble Classfiers

In [3]:
import random

from sklearn import metrics

#### Type 1: Genetic algorithm

In [5]:
'''
Helper functions
'''
# Temporary, for testing purposes only
def network_1(input):
    result = []
    limit = 1
    for i in range(0, 8):
        curr_prob = random.uniform(0, limit)
        result.append(curr_prob)
        limit -= curr_prob

    return result

# Temporary, for testing purposes only
def network_2(input):
    result = []
    limit = 1
    for i in range(0, 8):
        curr_prob = random.uniform(0, limit)
        result.append(curr_prob)
        limit -= curr_prob

    return result

def weighted_probability(input, curr_weight_combi):
    #TODO: Need to change to the final sequence of networks. This is temporary.
    network_1_output = network_1(input)
    result = [probability * curr_weight_combi[0] for probability in network_1_output]

    network_2_output = network_2(input)
    sum = 0

    for i in range(0, len(result)):
        result[i] += network_2_output[i] * curr_weight_combi[1]
        sum += result[i]

    for j in range(0, len(result)): # Normalising to ensure that the final output is still in terms of probability
        result[j] = result[j] / sum

    return result

def fitness(y_pred, y_true): # Negative log loss function
    return metrics.log_loss(y_true, y_pred)

def mutate(weight_combi): #TODO: Need to double-check if this is okay
    for i in range(0, len(weight_combi)):
        weight_combi[i] = weight_combi[i] * random.uniform(0.99, 1.01) # Randomly change by 2% (Not stated in the paper!)
    
    return weight_combi

def cross_over(num_of_networks, parent_1, parent_2): #TODO: Need to double-check if this is okay
    cut = random.randint(0, num_of_networks - 1)
    new_weight_combi = parent_1[:cut] 
    new_weight_combi.extend(parent_2[cut:])

    return new_weight_combi

def generate_possible_weight_combis(num_of_networks, num_of_combis, weight_limit):
    possible_weight_combis = []

    while (num_of_combis > 0):
        curr_weight_combi = []
        curr_combi_len = 0

        while (curr_combi_len < num_of_networks):
            curr_weight = random.uniform(0, weight_limit)
            curr_weight_combi.append(curr_weight)

            curr_combi_len += 1
        
        possible_weight_combis.append(curr_weight_combi)
        num_of_combis -= 1
    
    return possible_weight_combis


In [17]:
'''
Start of genetic algorithm
'''
# Defining essential variables
#TODO: Temporary for testing purposes
full_dataset_and_y_true = [('img1', [0, 0, 1, 0, 0, 0, 0, 0]), 
                           ('img2', [1, 0, 0, 0, 0, 0, 0, 0]), 
                           ('img3', [0, 0, 0, 0, 0, 0, 1, 0]), 
                           ('img4', [0, 1, 0, 0, 0, 0, 0, 0]), 
                           ('img5', [0, 0, 0, 0, 0, 1, 0, 0])]
num_of_networks = 2
num_of_required_weight_combis = 50
weight_limit = 1000
possible_weight_combis = generate_possible_weight_combis(num_of_networks, num_of_required_weight_combis, weight_limit)
max_num_of_iters = 5
# print(possible_weight_combis)

while (max_num_of_iters > 0):
    # Step 1: Randomly chossing 50% of the dataset to calculate the fitness scores for
    chosen_samples_and_y_true = {}
    required_num_of_samples = len(full_dataset_and_y_true)/2

    while required_num_of_samples > 0:
        curr_sample_and_y_true = random.choice(full_dataset_and_y_true)
        curr_sample = curr_sample_and_y_true[0]

        if (curr_sample not in chosen_samples_and_y_true):
            chosen_samples_and_y_true[curr_sample] = curr_sample_and_y_true[1] #TODO: Figure out how to obtain y_true
            required_num_of_samples -= 1

    # print(chosen_samples_and_y_true) # For testing

    # Step 2: Calculate the average fitness scores for each of the possible weight combinations
    fitness_and_weights = []

    for weights in possible_weight_combis:
        accumulated_fitness_score = 0
        num_of_samples = 0

        for sample in chosen_samples_and_y_true:
            y_pred = weighted_probability(sample, weights)
            y_true = chosen_samples_and_y_true[sample]
            fitness_score = fitness(y_pred, y_true)
            accumulated_fitness_score += fitness_score

            num_of_samples += 1
        
        avg_fitness_score = accumulated_fitness_score / num_of_samples
        fitness_and_weights.append((avg_fitness_score, weights))
    
    # print(fitness_and_weights) # For testing

    # Step 3: Rank the weight combis from best to worse
    fitness_and_weights.sort() # The combis with the lowest log loss is at the start
    # print(fitness_and_weights) # For testing

    # Step 4: Selecting parents
    parents = []
    curr_index = 0

    # Selecting top 20% of the weight combis
    top_20_percent = int(len(fitness_and_weights) / 5)
    while (top_20_percent > 0):
        parents.append(fitness_and_weights[curr_index][1])
        top_20_percent -= 1
        curr_index += 1

    # Randomly choosing another 10% of the weight combis
    another_10_percent = int(len(fitness_and_weights) / 10)
    while(another_10_percent > 0):
        random_score_and_parent = random.choice(fitness_and_weights[curr_index:])
        parents.append(random_score_and_parent[1])
        fitness_and_weights.remove(random_score_and_parent)

        another_10_percent -= 1
    
    # print(parents) # For testing

    # Step 5: Randomly mutate 5% of the selected parents
    num_of_parents_to_mutate = max(1, int(len(parents) / 10))
    index_of_parents_to_mutate = [random.randint(0, len(parents) - 1) for i in range(0, num_of_parents_to_mutate)]

    for index in index_of_parents_to_mutate:
        parents[index] = mutate(parents[index])
    
    # print(parents) # For testing

    # Step 6: Randomly cross over parents to produce new set of weight combinations
    new_weight_combis = []
    index_of_crossed_parents = []
    num_of_curr_weights = 0

    while (num_of_curr_weights < num_of_required_weight_combis):
        chosen_parents = (random.randint(0, len(parents) - 1), random.randint(0, len(parents) - 1))
        parent_1 = parents[chosen_parents[0]]
        parent_2 = parents[chosen_parents[1]]

        if (parent_1 != parent_2 and chosen_parents not in index_of_crossed_parents):
            new_weight_combi = cross_over(num_of_networks, parent_1, parent_2)
            new_weight_combis.append(new_weight_combi)
            num_of_curr_weights += 1

    possible_weight_combis = new_weight_combis
#     # print(possible_weight_combis) # For testing

    max_num_of_iters -= 1

# Step 7: Select the best weights combination
final_fitness_and_weights = []

for weights in possible_weight_combis:
    accumulated_fitness_score = 0
    num_of_samples = 0

    for sample_and_y_true in full_dataset_and_y_true:
        sample = sample_and_y_true[0]
        y_pred = weighted_probability(sample, weights)
        y_true = sample_and_y_true[1]
        fitness_score = fitness(y_pred, y_true)
        accumulated_fitness_score += fitness_score

        num_of_samples += 1
    
    avg_fitness_score = accumulated_fitness_score / num_of_samples
    final_fitness_and_weights.append((avg_fitness_score, weights))

final_fitness_and_weights.sort() # The combis with the lowest log loss is at the start
print("The best weight combination is: " + str(final_fitness_and_weights[0][1]))
print("The fitness score of this combination is: " + str(final_fitness_and_weights[0][0]))
# return final_fitness_and_weights[0][1]

The best weight combination is: [378.42656747361457, 732.1775902888443]
The fitness score of this combination is: 0.3554262856466003


In [22]:
best_weights = final_fitness_and_weights[0][1]

def final_outcome(input):
    final_probability = weighted_probability(input, best_weights)

    highest_value_and_class = [final_probability[0], 0]

    for i in range(1, len(final_probability)):
        if (final_probability[i] > highest_value_and_class[0]):
            highest_value_and_class[0] = final_probability[i]
            highest_value_and_class[1] = i

    return highest_value_and_class[1] # Returns the class

for img in full_dataset_and_y_true:
    print(best_weights)
    print(final_outcome(img))

[378.42656747361457, 732.1775902888443]
0
[378.42656747361457, 732.1775902888443]
0
[378.42656747361457, 732.1775902888443]
1
[378.42656747361457, 732.1775902888443]
0
[378.42656747361457, 732.1775902888443]
0


#### Type 2: Majority Vote

In [82]:
'''
Helper functions
'''
# Temporary, for testing purposes only
def network_1(input):
    result = []
    limit = 1
    for i in range(0, 8):
        curr_prob = random.uniform(0, limit)
        result.append(curr_prob)
        limit -= curr_prob

    return result

# Temporary, for testing purposes only
def network_2(input):
    result = []
    limit = 1
    for i in range(0, 8):
        curr_prob = random.uniform(0, limit)
        result.append(curr_prob)
        limit -= curr_prob

    return result

def final_outcome(softmax_outcome):
    highest_value_and_class = [softmax_outcome[0], 0]

    for i in range(1, len(softmax_outcome)):
        if (softmax_outcome[i] > highest_value_and_class[0]):
            highest_value_and_class[0] = softmax_outcome[i]
            highest_value_and_class[1] = i

    return highest_value_and_class[1]
    
'''
Start of majority vote algorithm
'''
# Defining essential variables
#TODO: Temporary for testing purposes
full_dataset= ['img1', 'img2', 'img3', 'img4', 'img5']

for img in full_dataset:
    class_and_votes = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}

    network_1_outcome = network_1(img)
    class_and_votes[final_outcome(network_1_outcome)] += 1

    network_2_outcome = network_2(img)
    class_and_votes[final_outcome(network_2_outcome)] += 1

    # print(class_and_votes) # For testing
    print(max(class_and_votes, key=class_and_votes.get))
    # return max(class_and_votes, key=class_and_votes.get)


{0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
0
{0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
0
{0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
0
{0: 2, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
0
{0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
0


#### Type 3: Random forests